In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.jmt.*
import jline.solvers.mva.*
import jline.solvers.mam.*
import jline.util.matrix.*
import kotlin.math.*

In [ ]:
val model = Network("model")

// Create 5 multi-server queues with PS scheduling
val nodes = Array<Node?>(5) { null }
for (i in 0 until 5) {
    nodes[i] = Queue(model, "Queue${i+1}", SchedStrategy.PS)
    (nodes[i] as Queue).numberOfServers = i + 1  // 1,2,3,4,5 servers
}

val source = Source(model, "Source")
val sink = Sink(model, "Sink")

// Create job classes
val closedClass = ClosedClass(model, "ClosedClass", 3, nodes[0], 0)
val openClass = OpenClass(model, "OpenClass", 0)

// Set service rates
for (i in 0 until 5) {
    nodes[i]!!.setService(closedClass, Exp((i + 1).toDouble()))
    nodes[i]!!.setService(openClass, Exp(sqrt((i + 1).toDouble())))
}

// Set arrival rate for open class
source.setArrival(openClass, Exp(0.3))

println("Created 5 multi-server PS queues with ${(0..4).map { i -> "Queue${i+1}(${i+1} servers)" }.joinToString(", ")}")

In [ ]:
// Setup routing matrix
val M = model.numberOfStations
val K = model.numberOfClasses

val P = model.initRoutingMatrix()

// Closed class routing: Queue1 -> Queue2 -> Queue3 -> Queue4 -> Queue1 (circular)
P.set(closedClass, closedClass, nodes[0], nodes[1], 1.0)
P.set(closedClass, closedClass, nodes[1], nodes[2], 1.0)
P.set(closedClass, closedClass, nodes[2], nodes[3], 1.0)
P.set(closedClass, closedClass, nodes[3], nodes[0], 1.0)

// Open class routing: Source -> Queue1 -> Queue2 -> Queue3 -> Queue5 -> Sink
P.set(openClass, openClass, source, nodes[0], 1.0)
P.set(openClass, openClass, nodes[0], nodes[1], 1.0)
P.set(openClass, openClass, nodes[1], nodes[2], 1.0)
P.set(openClass, openClass, nodes[2], nodes[4], 1.0)
P.set(openClass, openClass, nodes[4], sink, 1.0)

model.link(P)

println("Routing configured:")
println("  Closed class: Queue1->Queue2->Queue3->Queue4->Queue1 (excludes Queue5)")
println("  Open class: Source->Queue1->Queue2->Queue3->Queue5->Sink (excludes Queue4)")

In [ ]:
// Create solvers with appropriate configurationsval solvers = mutableListOf<NetworkSolver>()// Add solvers (some may not be available or may fail)try {    solvers.add(CTMC(model, "cutoff", 3))  // CTMC with cutoff} catch (e: Exception) {    println("CTMC solver not available: ${e.message}")}try {    solvers.add(JMT(model, "samples", 100000, "seed", 23000))} catch (e: Exception) {    println("JMT solver not available: ${e.message}")}// SSA solver (commented to match MATLAB)// try {//     solvers.add(SSA(model, "cutoff", 3, "seed", 23000))// } catch (e: Exception) {//     println("SSA solver not available: ${e.message}")// }try {    solvers.add(MVA(model, "method", "exact"))} catch (e: Exception) {    println("MVA solver not available: ${e.message}")}try {    solvers.add(MAM(model, "seed", 23000, "keep", false, "verbose", true))} catch (e: Exception) {    println("MAM solver not available: ${e.message}")}println("\nAvailable solvers: ${solvers.size}")

In [ ]:
// Solve and display resultssolvers.forEachIndexed { index, solver ->    try {        println("\n${"+".repeat(50)}")                println("${"+".repeat(50)}")                val avgTable = solver.avgTable        avgTable.print()            } catch (e: Exception) {        println("\nSOLVER ${solver.name.replace("Solver", "")} FAILED: ${e.message}")    }}

## Results Analysis

This mixed queueing network with PS scheduling demonstrates:

1. **Processor Sharing**: Resources shared equally among jobs at each queue
2. **Class Separation**: Different routing for closed and open classes
3. **Multi-Server Capacity**: Each queue can serve multiple jobs simultaneously
4. **Solver Comparison**: Different numerical methods for validation

**PS vs FCFS Differences:**
- **Service Discipline**: PS shares server capacity equally among all jobs
- **Response Times**: Generally more predictable under PS
- **Fairness**: PS provides better fairness for jobs of different sizes

**Performance Insights:**
- Compare utilizations and response times between classes
- Observe the effect of different service rates (linear vs square root)
- Note how multi-server capacity affects overall system performance